In [61]:
import pandas as pd
import numpy as np

#read each of the .csv tables
#validated these are correct
fl_equ = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioid-prescription-regulations-team-4/master/20_intermediate_files/FL_opioid_equ.csv?token=AMVJ2QKKKXA2KERBXWYKPNC5ZVR7K')
ga_equ = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioid-prescription-regulations-team-4/master/20_intermediate_files/GA_opioid_equ.csv?token=AMVJ2QNATVTFJ76YJUVZMUC5ZVSBE')
counties = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioid-prescription-regulations-team-4/master/20_intermediate_files/Counties_DF_Merge_Ready_v2.csv?token=AMVJ2QODBV4X3BIX6NZLGPK5ZVR3M')
deaths = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioid-prescription-regulations-team-4/master/20_intermediate_files/overdose_deaths.csv?token=AMVJ2QP2BOWXM27YNZG2MJ25ZVSDA')
controls = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioid-prescription-regulations-team-4/master/20_intermediate_files/State_Opioid_Controls_Year_Changed.csv?token=AMVJ2QO3PKYYKA6DGMJHH6K5ZVSJA')

In [62]:
#read each of the .csv tables
nc_equ = pd.read_csv('NC_opioid_equ.csv')
al_equ = pd.read_csv('AL_opioid_equ.csv')
fl_equ = pd.read_csv('FL_opioid_equ.csv')
ga_equ = pd.read_csv('GA_opioid_equ.csv')
counties = pd.read_csv('population_extra_v2.csv')
deaths = pd.read_csv('overdose_deaths.csv')
controls = pd.read_csv('State_opioid_Controls_Year_Changed.csv')

In [63]:
#With new population data, need to add in state data for 2008
counties.loc[counties['State'] == 'AL', 'State_Full'] = 'Alabama'
counties.loc[counties['State'] == 'GA', 'State_Full'] = 'Georgia'
counties.loc[counties['State'] == 'FL', 'State_Full'] = 'Florida'
counties.loc[counties['State'] == 'NC', 'State_Full'] = 'North Carolina'
counties.loc[counties['State'] == 'CA', 'State_Full'] = 'California'
counties.loc[counties['State'] == 'TX', 'State_Full'] = 'Texas'
counties.loc[counties['State'] == 'AZ', 'State_Full'] = 'Arizona'
counties.loc[counties['State'] == 'NM', 'State_Full'] = 'New Mexico'
counties.loc[counties['State'] == 'WA', 'State_Full'] = 'Washington'
counties.loc[counties['State'] == 'OR', 'State_Full'] = 'Oregon'
counties.loc[counties['State'] == 'ID', 'State_Full'] = 'Idaho'
counties.loc[counties['State'] == 'MO', 'State_Full'] = 'Montana'

counties.loc[counties['State_Full'] == 'Alabama', 'State'] = 'AL'
counties.loc[counties['State_Full'] == 'Georgia', 'State'] = 'GA'
counties.loc[counties['State_Full'] == 'Florida', 'State'] = 'FL'
counties.loc[counties['State_Full'] == 'North Carolina', 'State'] = 'NC'
counties.loc[counties['State_Full'] == 'California', 'State'] = 'CA'
counties.loc[counties['State_Full'] == 'Texas', 'State'] = 'TX'
counties.loc[counties['State_Full'] == 'Arizona', 'State'] = 'AZ'
counties.loc[counties['State_Full'] == 'New Mexico', 'State'] = 'NM'
counties.loc[counties['State_Full'] == 'Washington', 'State'] = 'WA'
counties.loc[counties['State_Full'] == 'Oregon', 'State'] = 'OR'
counties.loc[counties['State_Full'] == 'Idaho', 'State'] = 'ID'
counties.loc[counties['State_Full'] == 'Montana', 'State'] = 'MO'




In [64]:
#Clean county data
counties['County'] = counties['County'].replace(regex = True, to_replace = [' County', ' Borough', ' Parish', ' City'], value = '')
counties['County'] = counties['County'].replace(regex = True, to_replace = ['St. ', 'St'], value = 'Saint ')
counties['County'] = counties['County'].str.upper()
counties['Year'] = counties['Year'].astype(int)

In [65]:
#confirmed 2008 is correct
counties.loc[(counties['Year'] == 2008)].sample(10)

,Year,State_Full,State,County,Population
24155,2008,Georgia,GA,GLASCOCK,3010
24171,2008,Georgia,GA,JACKSON,59420
24139,2008,Georgia,GA,DOOLY,14191
24361,2008,NaN,MS,SMITH,16345
24534,2008,Oregon,OR,CURRY,22305
24829,2008,Washington,WA,GRANT,84944
24703,2008,Texas,TX,LAMPASAS,19812
24335,2008,NaN,MS,LAWRENCE,13038
24387,2008,Montana,MO,BOLLINGER,12429
23976,2008,California,CA,DEL NORTE,28496


In [66]:
#Dekalb was a problem county, changed name in counties to be without a space
counties = counties.replace(to_replace = 'DE KALB', value = 'DEKALB')

In [67]:
counties['State'].value_counts().sum()

8402

In [68]:
counties_wash = counties.loc[counties['State'] == "WA"]
counties_wash.tail()

,Year,State_Full,State,County,Population
24851,2008,Washington,WA,WAHKIAKUM,4033
24852,2008,Washington,WA,WALLA WALLA,57315
24853,2008,Washington,WA,WHATCOM,196708
24854,2008,Washington,WA,WHITMAN,43802
24855,2008,Washington,WA,YAKIMA,235272


In [69]:
deaths.sample(20)

,index,State,County,Year,Deaths
439,30,CA,Placer County,2004.0,20.0
6859,484,OH,Greene County,2014.0,40.0
7128,2,AK,Matanuska-Susitna Borough,2015.0,13.0
941,79,FL,Duval County,2005.0,102.0
658,249,NH,Hillsborough County,2004.0,28.0
6602,227,KS,Johnson County,2014.0,53.0
1868,525,WV,Mercer County,2006.0,16.0
1608,265,NC,Brunswick County,2006.0,17.0
6287,618,TX,Tarrant County,2013.0,148.0
4787,494,OK,Washington County,2011.0,11.0


In [70]:
controls.head()

,State_Full,Year_Changed,State
0,California,Control,CA
1,Arizona,Control,AZ
2,New Mexico,Control,NM
3,Texas,2007,TX
4,Georgia,Control,GA


In [71]:
#merge the counties and population data with the control states table
outer_counties = controls.merge(counties, how = 'outer', on = ['State', 'State_Full'], indicator = True )

In [72]:
#as expected, only right or both merges, no left merges
outer_counties.head()
outer_counties.sample(50)
#outer_counties.describe()
#outer_counties._merge.value_counts()

,State_Full,Year_Changed,State,Year,County,Population,_merge
17643,North Carolina,NaN,NC,2003.0,JONES,10183.0,right_only
10947,Kansas,NaN,NaN,2009.0,NEMAHA,10118.0,right_only
5440,Florida,2010,FL,2007.0,SAINT LUCIE,260461.0,both
20351,Pennsylvania,NaN,NaN,2005.0,WAYNE,50158.0,right_only
22870,Virginia,NaN,NaN,2004.0,SURRY,6990.0,right_only
4360,Georgia,Control,GA,2015.0,WALTON,88399.0,both
18605,North Dakota,NaN,NaN,2004.0,LAMOURE,4490.0,right_only
4917,Alabama,Control,AL,2009.0,TALLAPOOSA,40778.0,both
167,California,Control,CA,2005.0,TEHAMA,59702.0,both
13490,Michigan,NaN,NaN,2015.0,ALLEGAN,114625.0,right_only


In [73]:
outer_counties.loc[outer_counties['_merge'] == 'left_only']

,State_Full,Year_Changed,State,Year,County,Population,_merge
5116,Mississippi,Control,MS,NaN,NaN,NaN,left_only
6512,Montana,Control,MT,NaN,NaN,NaN,left_only


In [74]:
#outer_counties['Year'].dtypes

In [75]:
#Clean the death data table to replace County, Borough, Parish, and city with '' and make them upper case
deaths['County'] = deaths['County'].replace(regex = True, to_replace = [' County', ' Borough', ' Parish', ' City'], value = '')
deaths['County'] = deaths['County'].replace(regex = True, to_replace = ['St. '], value = 'Saint ')
deaths['County'] = deaths['County'].str.upper()
deaths['Year'] = deaths['Year'].astype(int)
#Confirmed the changes were correct

In [76]:
deaths.sample(10)
#outer_counties.sample(10)

,index,State,County,Year,Deaths
5182,195,IN,ELKHART,2012,15.0
6797,422,NM,BERNALILLO,2014,193.0
5677,8,AL,ETOWAH,2013,19.0
3849,209,KS,WYANDOTTE,2010,16.0
4410,117,FL,MARTIN,2011,22.0
1419,76,FL,BAY,2006,34.0
4643,350,NC,DAVIDSON,2011,30.0
435,26,CA,MARIN,2004,16.0
6844,469,OH,BROWN,2014,17.0
1894,22,AZ,NAVAJO,2007,15.0


In [77]:
#drop indicator column
outer_counties = outer_counties.drop(axis = 1, columns = ['_merge', 'State_Full'])

In [78]:
death_added = deaths.merge(outer_counties, how = 'outer', on = ['State', 'Year', 'County'], indicator = True)
death_added._merge.value_counts()
#death_added.sample(10)
#This resulted in less left_only joins than expected, but this is because not all counties have death data for drug overdoses

right_only    22224
left_only      5186
both           2734
Name: _merge, dtype: int64

In [79]:
death_added['Year'].value_counts().sum()

30142

In [80]:
death_added.sample(20)

,index,State,County,Year,Deaths,Year_Changed,Population,_merge
16226,NaN,NaN,HARTFORD,2006.0,NaN,NaN,872945.0,right_only
14388,NaN,NaN,UNION,2006.0,NaN,NaN,28064.0,right_only
10816,NaN,GA,MERIWETHER,2009.0,NaN,Control,22858.0,right_only
27292,NaN,NaN,ALLEGHENY,2005.0,NaN,NaN,1234319.0,right_only
24209,NaN,MO,CARTER,2007.0,NaN,NaN,1234.0,right_only
18082,NaN,NaN,SAINT CLAIR,2014.0,NaN,NaN,265729.0,right_only
16742,NaN,NaN,CLINTON,2004.0,NaN,NaN,36107.0,right_only
4002,362.0,NC,SURRY,2010.0,16.0,NaN,73694.0,both
10421,NaN,GA,RANDOLPH,2005.0,NaN,Control,7342.0,right_only
28013,NaN,NaN,CORSON,2003.0,NaN,NaN,4303.0,right_only


In [81]:
#rename columns to match with the intermediate merged table for Florida
fl_equ = fl_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
fl_equ.head()

,Year,FIPS,County,Morphine_equ,State
0,2006,12001,ALACHUA,82596.618688,FL
1,2006,12003,BAKER,11900.178638,FL
2,2006,12005,BAY,61998.102472,FL
3,2006,12007,BRADFORD,13424.830895,FL
4,2006,12009,BREVARD,201186.153403,FL


In [82]:
#rename columns to match with the intermediate merged table for Georgia
ga_equ = ga_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
ga_equ.head()

,Year,FIPS,County,Morphine_equ,State
0,2006,13001,APPLING,3996.843779,GA
1,2006,13003,ATKINSON,1375.483686,GA
2,2006,13005,BACON,4586.778211,GA
3,2006,13007,BAKER,9.651600,GA
4,2006,13009,BALDWIN,11069.705010,GA


In [83]:
#drop indicator column from death_added
death_added = death_added.drop(axis = 1, columns = '_merge')

In [84]:
#merge florida morphine and georgia morphine
#Expect only left or right merges
ga_fl_combined = ga_equ.merge(fl_equ, how = 'outer', on = ['County', 'State', 'Morphine_equ','FIPS', 'Year'], indicator = True)
ga_fl_combined.sample(10)
#ga_fl_combined._merge.value_counts()

,Year,FIPS,County,Morphine_equ,State,_merge
1411,2011,12035,FLAGLER,5.850027e+04,FL,right_only
627,2010,13039,CAMDEN,2.268313e+04,GA,left_only
1089,2006,12057,HILLSBOROUGH,1.027724e+06,FL,right_only
1332,2010,12009,BREVARD,5.088410e+05,FL,right_only
1092,2006,12063,JACKSON,9.866875e+03,FL,right_only
676,2010,13145,HARRIS,1.156425e+03,GA,left_only
230,2007,13161,JEFF DAVIS,3.496705e+03,GA,left_only
1354,2010,12055,HIGHLANDS,4.572984e+04,FL,right_only
199,2007,13097,DOUGLAS,4.246226e+04,GA,left_only
267,2007,13237,PUTNAM,3.344067e+03,GA,left_only


In [85]:
#rename alabama and north carolina columns

al_equ = al_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
al_equ['State'] = 'AL'
al_equ.head()

nc_equ = nc_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
nc_equ['State'] = 'NC'
nc_equ.head()

,Year,County,Morphine_equ,State
0,2006,ALAMANCE,36524.619625,NC
1,2006,ALEXANDER,11079.462527,NC
2,2006,ALLEGHANY,2412.194188,NC
3,2006,ANSON,3869.386819,NC
4,2006,ASHE,7183.591413,NC


In [86]:
#combine alabama and north carolina, should only have left and right merges

In [87]:
al_nc_combined = al_equ.merge(nc_equ, how = 'outer', on = ['County', 'State', 'Morphine_equ','Year'], indicator = True)
al_nc_combined.sample(10)

,Year,County,Morphine_equ,State,_merge
426,2010,SHELBY,11164.785563,AL,left_only
1284,2012,SCOTLAND,32550.396840,NC,right_only
160,2007,SCIOTO,67323.677239,AL,left_only
614,2012,WOOD,32754.979019,AL,left_only
815,2008,ANSON,5351.957942,NC,right_only
574,2012,LORAIN,95499.770065,AL,left_only
998,2009,VANCE,35636.909655,NC,right_only
388,2010,HOCKING,11047.822658,AL,left_only
94,2007,BELMONT,34814.552577,AL,left_only
48,2006,MADISON,13351.237638,AL,left_only


In [88]:
#drop merge columns
al_nc_combined = al_nc_combined.drop(columns = '_merge')


In [89]:
ga_fl_combined = ga_fl_combined.drop(columns = '_merge')


In [90]:
#combine all intermediate drug tables

In [91]:
m_combined = al_nc_combined.merge(ga_fl_combined, how = 'outer', on = ['County', 'State', 'Morphine_equ','Year'], indicator = True)
m_combined.sample(10)

,Year,County,Morphine_equ,State,FIPS,_merge
2058,2010,WILCOX,2472.253399,GA,13315.0,right_only
2163,2011,MUSCOGEE,95051.734524,GA,13217.0,right_only
2319,2012,PAULDING,42966.593925,GA,13223.0,right_only
2665,2010,LEON,56268.735998,FL,12073.0,right_only
1888,2009,THOMAS,8973.985617,GA,13275.0,right_only
2329,2012,RICHMOND,99701.892823,GA,13247.0,right_only
451,2011,CLARK,53500.267625,AL,NaN,left_only
1448,2006,WAYNE,9923.329898,GA,13305.0,right_only
2804,2012,MARION,175709.775372,FL,12083.0,right_only
2119,2011,GILMER,20971.320466,GA,13123.0,right_only


In [92]:
#m_combined.head()

In [93]:
#death_added.sample(10)

In [94]:
#check merge
#Validated, similar to expected counts
m_combined._merge.value_counts()

right_only    1529
left_only     1302
both             0
Name: _merge, dtype: int64

In [95]:
#drop _merge column
m_combined = m_combined.drop(axis = 1, columns = '_merge')

In [96]:
#add the morphine table into the intermediate table
morphine_added = m_combined.merge(death_added, how = 'outer', on = ['County', 'State','Year'], indicator = True)
morphine_added.describe()
morphine_added._merge.value_counts()
#morphine_added.loc[:, 'Year'].value_counts()

right_only    28437
both           1707
left_only      1124
Name: _merge, dtype: int64

In [97]:
#We are not going to use Georgia Anyway for morphine checks
#morphine_added_check = morphine_added.loc[((morphine_added['County'] == 'MUSCOGEE'))]
#morphine_added_check.head()

In [98]:
#morphine_added[morphine_added._merge == 'left_only']

In [99]:
#morphine_added.head()

In [100]:
#morphine_added._merge.value_counts()

In [101]:
#remove merge column
#morphine_added = morphine_added.drop(axis = 1, columns = '_merge')

In [102]:
#Drop the unnamed and index column, which was an index column left over from the import of an intermediate table
final_frame = morphine_added.drop(columns = '_merge')
final_frame = final_frame.drop(columns = 'index')
final_frame.sample(10)

,Year,County,Morphine_equ,State,FIPS,Deaths,Year_Changed,Population
23030,2015.0,CUMBERLAND,NaN,NaN,NaN,NaN,NaN,246338.0
6450,2010.0,CARTER,NaN,OK,NaN,11.0,NaN,NaN
1712,2008.0,OCONEE,4502.584808,GA,13219.0,NaN,Control,31697.0
27877,2013.0,LICKING,NaN,NaN,NaN,NaN,NaN,168375.0
22695,2007.0,TERREBONNE,NaN,NaN,NaN,NaN,NaN,108627.0
7144,2011.0,LIBERTY,NaN,TX,NaN,10.0,2007,76206.0
4565,2006.0,WILLIAMSON,NaN,TX,NaN,23.0,2007,350113.0
877,2008.0,ORANGE,21789.929192,NC,NaN,NaN,NaN,129764.0
12431,2004.0,TIFT,NaN,GA,NaN,NaN,Control,40204.0
9765,2015.0,FAYETTE,NaN,OH,NaN,16.0,NaN,NaN


In [103]:
final_frame.sample()

,Year,County,Morphine_equ,State,FIPS,Deaths,Year_Changed,Population
31219,2008.0,JEFFERSON,NaN,MS,NaN,NaN,NaN,7927.0


In [104]:
#Replace Na's with zero for death data and morphine equivalents
final_frame['Morphine_equ'] = final_frame['Morphine_equ'].fillna(value = 0)
final_frame['Deaths'] = final_frame['Deaths'].fillna(value = 0)

In [105]:
#add in a deaths per capita column
final_frame['Deaths_PC'] = final_frame['Deaths'] / final_frame['Population']
final_frame.sample(10)

,Year,County,Morphine_equ,State,FIPS,Deaths,Year_Changed,Population,Deaths_PC
11433,2007.0,COLEMAN,0.000000,TX,NaN,0.0,2007,8492.0,0.00000
14129,2003.0,BOONE,0.000000,NaN,NaN,0.0,NaN,49550.0,0.00000
26978,2014.0,ERIE,0.000000,NaN,NaN,0.0,NaN,75828.0,0.00000
7351,2012.0,DEARBORN,0.000000,IN,NaN,11.0,NaN,NaN,NaN
1484,2007.0,CLAYTON,37904.796417,GA,13063.0,19.0,Control,272705.0,0.00007
21501,2006.0,SEWARD,0.000000,NaN,NaN,0.0,NaN,22612.0,0.00000
25070,2004.0,CAMDEN,0.000000,NaN,NaN,0.0,NaN,515620.0,0.00000
31170,2014.0,ARECIBO MUNICIPIO,0.000000,NaN,NaN,0.0,NaN,91540.0,0.00000
27912,2014.0,RICHLAND,0.000000,NaN,NaN,0.0,NaN,121942.0,0.00000
24845,2006.0,PEARL RIVER,0.000000,NaN,NaN,0.0,NaN,56237.0,0.00000


In [106]:
#Add morphine per capita to frame
final_frame['Morphine_Equivalents_PC'] = final_frame['Morphine_equ']/ final_frame['Population']
final_frame.sample(10)

,Year,County,Morphine_equ,State,FIPS,Deaths,Year_Changed,Population,Deaths_PC,Morphine_Equivalents_PC
14412,2003.0,MONTGOMERY,0.000000,NaN,NaN,0.0,NaN,35053.0,0.000000,0.000000
1535,2007.0,JOHNSON,2149.062204,GA,13167.0,0.0,Control,9414.0,0.000000,0.228284
27420,2006.0,DICKEY,0.000000,NaN,NaN,0.0,NaN,5377.0,0.000000,0.000000
7819,2013.0,MARSHALL,0.000000,AL,NaN,13.0,Control,94760.0,0.000137,0.000000
5615,2009.0,PULASKI,0.000000,AR,NaN,28.0,NaN,NaN,NaN,NaN
19041,2007.0,SCHUYLER,0.000000,NaN,NaN,0.0,NaN,18923.0,0.000000,0.000000
1765,2009.0,BARROW,28692.509040,GA,13013.0,0.0,Control,66458.0,0.000000,0.431739
31140,2011.0,CAROLINA MUNICIPIO,0.000000,NaN,NaN,0.0,NaN,175129.0,0.000000,0.000000
24033,2012.0,MIDLAND,0.000000,NaN,NaN,0.0,NaN,83822.0,0.000000,0.000000
10094,2004.0,MONO,0.000000,CA,NaN,0.0,Control,12687.0,0.000000,0.000000


In [107]:
final_frame.loc[:,'Year'].value_counts()
final_frame.loc[:,'County'].value_counts()

WASHINGTON                 341
JEFFERSON                  304
FRANKLIN                   247
JACKSON                    233
MADISON                    219
                          ... 
MOCA MUNICIPIO               1
HORMIGUEROS MUNICIPIO        1
STARR                        1
SAINT JAMES                  1
SAN SEBASTI·N MUNICIPIO      1
Name: County, Length: 2011, dtype: int64

In [108]:
#export the final frame to a csv
#final_frame.to_csv('Final_Data_Frame_Python_Midterm.csv')

In [109]:
#Check filter to see if it is reasonable

In [110]:
ultimate_final_frame = final_frame.loc[(final_frame['State'] == 'CA') | (final_frame['State'] == 'AZ') | (final_frame['State'] == 'NM') | (final_frame['State'] == 'TX') | (final_frame['State'] == 'GA') | (final_frame['State'] == 'AL') | (final_frame['State'] == 'NC') | (final_frame['State'] == 'FL') | (final_frame['State'] == 'OR')| (final_frame['State'] == 'ID') | (final_frame['State'] == 'MT') | (final_frame['State'] == 'WA')]
ultimate_final_frame.sample(10)

,Year,County,Morphine_equ,State,FIPS,Deaths,Year_Changed,Population,Deaths_PC,Morphine_Equivalents_PC
498,2011.0,MORROW,7636.801999,AL,NaN,0.0,NaN,NaN,NaN,NaN
4599,2006.0,YAKIMA,0.000000,WA,NaN,12.0,2012,230139.0,0.000052,0.000000
6107,2010.0,TUSCALOOSA,0.000000,AL,NaN,22.0,Control,195036.0,0.000113,0.000000
12305,2003.0,WHEELER,0.000000,GA,NaN,0.0,Control,6619.0,0.000000,0.000000
1633,2008.0,CHEROKEE,57124.916040,GA,13057.0,16.0,Control,208271.0,0.000077,0.274282
5866,2009.0,CHAVES,0.000000,NM,NaN,13.0,Control,62365.0,0.000208,0.000000
8574,2014.0,SAN DIEGO,0.000000,CA,NaN,436.0,Control,3263431.0,0.000134,0.000000
1982,2010.0,HOUSTON,58661.750919,GA,13153.0,0.0,Control,140713.0,0.000000,0.416889
11375,2006.0,UVALDE,0.000000,TX,NaN,0.0,2007,26520.0,0.000000,0.000000
13809,2006.0,KLICKITAT,0.000000,WA,NaN,0.0,2012,19780.0,0.000000,0.000000


In [111]:
#Sort by year, ascending
ultimate_final_frame = ultimate_final_frame.sort_values('Year')

In [112]:
ultimate_final_frame.to_csv('2.0_ultimate_final_frame.csv')